### IMPORTING DATASETS

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds

In [2]:
os.getcwd()

os.chdir("C:\\Users\\DSAMIT\\TEST\\Recommendation_Engine_Module")

df_movies = pd.read_csv("movies_clean.csv")

df_ratings = pd.read_csv("reviews_clean.csv")

df_movies = df_movies.iloc[:,1:]
df_ratings=df_ratings.iloc[:,1:]

In [3]:
df_mr = pd.merge(df_ratings,df_movies.loc[:,["movie_id","movie","genre"]],how = "left",on="movie_id")

In [4]:
df_new = df_mr.loc[:,["user_id","rating","movie"]]

In [5]:
df_new["user_id"].nunique()

55789

In [6]:
df_new100=df_new.loc[df_new["user_id"]<=100,:] #taking only 100 users

In [7]:
df_new100_R=df_new100.pivot(index = "user_id",columns = "movie",values = "rating").fillna(0)

df_new100_mat = df_new100_R.as_matrix()##converting into matrix for svd

C:\Users\DSAMIT\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
def apply_svd(k):
    U, sigma, Vt=svds(df_new100_mat,k)
    sigma = np.diag(sigma)
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
    return all_user_predicted_ratings

In [9]:
df_svd = pd.DataFrame(apply_svd(25),columns = df_new100_R.columns)

In [10]:
user_id=pd.Series(list(range(1,101,1)))

df_svd_new = pd.concat([user_id,df_svd],axis=1).rename(columns = {0:"user_id"})

In [11]:
df_svd_new.head()

,user_id,12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),2 Guns (2013),2001: A Space Odyssey (1968),20th Century Women (2016),21 Jump Street (2012),22 Jump Street (2014),3 Idiots (2009),...,You're Next (2011),You're Not You (2014),Young Adult (2011),Zamani barayé masti asbha (2000),Zendegi-e Khosousi-e Agha Va Khanom-e Mim (2012),Zero Dark Thirty (2012),Zodiac (2007),Zoolander (2001),Zoolander 2 (2016),xXx (2002)
0,1,-8.326355e-03,-8.110858e-02,-1.021166e-02,-9.012270e-02,3.246242e-04,-4.768306e-03,1.014891e-01,-9.849866e-02,-7.596068e-02,...,2.142297e-03,1.733268e-17,2.885549e-04,-3.708682e-03,-1.589435e-03,9.055263e-03,7.922764e-02,-1.285615e-03,-3.887993e-03,4.173162e-03
1,2,1.039256e+01,2.558558e-02,5.124841e-04,-5.659740e-02,1.407735e-04,3.639586e-04,-3.974307e-02,3.593072e-02,3.507458e-02,...,-8.486194e-04,-2.610582e-16,1.251320e-04,2.830789e-04,1.213195e-04,-3.398369e-02,5.258358e-03,-1.740651e-02,1.437309e-01,-8.977774e-03
2,3,-1.637022e-02,1.458952e-01,-1.931053e-02,1.070135e-02,3.814509e-04,-2.281580e-03,-7.438640e-02,9.518509e-02,-2.119558e-02,...,1.058839e-05,3.434144e-18,3.390675e-04,-1.774562e-03,-7.605266e-04,7.060074e-02,-1.189207e-02,3.543378e-03,2.552496e-03,4.666808e-02
3,4,-4.506112e-02,-1.416855e-02,-3.543361e-02,2.160812e-01,-3.698316e-05,8.563210e-04,8.534996e-02,-4.519787e-02,2.880248e-01,...,-9.129552e-06,1.171165e-17,-3.287392e-05,6.660275e-04,2.854403e-04,3.236381e-02,-3.759160e-03,-1.003865e-02,1.600233e-03,2.688620e-02
4,5,2.774719e-16,1.811097e-16,-3.232256e-17,-5.660500e-17,-6.058540e-16,-8.245116e-17,2.696284e-16,5.036609e-17,6.779830e-17,...,3.165489e-18,9.640163e-34,-5.385369e-16,-6.412868e-17,-2.748372e-17,-1.762828e-16,1.250836e-16,8.994341e-17,-2.704683e-18,-1.127678e-16


In [12]:
def top_movie_recomender(user_ID):
    
    user_id=user_ID - 1
    
    orig_ratings=pd.DataFrame(df_new100_R.iloc[25].sort_values(ascending = False)).reset_index()
    orig_ratings.columns = ["movie","original_ratings"]
    final1=pd.merge(orig_ratings,df_mr[["movie","genre"]],how= "inner",on="movie").drop_duplicates()

    pred_ratings=pd.DataFrame(df_svd_new.iloc[25][1:].sort_values(ascending = False)).reset_index()
    pred_ratings.columns = ["movie","predicted_ratings"]
    final2=pd.merge(pred_ratings,df_mr[["movie","genre"]],how= "inner",on="movie").drop_duplicates()
    
    return final1,final2

In [13]:
df_orig,df_pred=top_movie_recomender(25)

In [14]:
df_orig.head(10)

,movie,original_ratings,genre
0,Hacksaw Ridge (2016),10.0,Biography|Drama|War
1622,Mystic River (2003),10.0,Crime|Drama|Mystery
1914,21 Jump Street (2012),10.0,Action|Comedy|Crime
2251,The Prestige (2006),10.0,Drama|Mystery|Sci-Fi
2890,Shutter Island (2010),10.0,Mystery|Thriller
3427,The Breakfast Club (1985),9.0,Comedy|Drama
3738,Million Dollar Arm (2014),9.0,Biography|Drama|Sport
3907,22 Jump Street (2014),9.0,Action|Comedy|Crime
4714,Snowden (2016),9.0,Biography|Drama|Thriller
5156,Cardboard Boxer (2016),8.0,Drama


In [15]:
df_pred.head(10)

,movie,predicted_ratings,genre
0,21 Jump Street (2012),10.002623,Action|Comedy|Crime
337,The Prestige (2006),10.001696,Drama|Mystery|Sci-Fi
976,Hacksaw Ridge (2016),9.997631,Biography|Drama|War
2598,Shutter Island (2010),9.990726,Mystery|Thriller
3135,Mystic River (2003),9.988212,Crime|Drama|Mystery
3427,The Breakfast Club (1985),9.004561,Comedy|Drama
3738,22 Jump Street (2014),9.000753,Action|Comedy|Crime
4545,Snowden (2016),8.989391,Biography|Drama|Thriller
4987,Million Dollar Arm (2014),8.989391,Biography|Drama|Sport
5156,Jack Reacher (2012),8.023369,Action|Crime|Mystery
